## PDF A JSON

In [ ]:
# import PyPDF2
# import re
# import json


# def extraer_info_desde_texto(texto):
#     # Expresiones regulares para encontrar capítulos y artículos
#     # regex_capitulo = re.compile(r'CAPÍTULO ([IVXLC]+): (.+)')
#     regex_capitulo = re.compile(r'.*CAP[IÍ]TULO ([IVXLC]+):.*')
#     # regex_articulo = re.compile(r'Art[ií]culo (\d+)\. – (.+)')
#     # regex_articulo = re.compile(r'.*(Artículo).*')
    
#     info = []
#     capitulo_actual = None
#     articulo_actual = None
#     texto_actual = ''

    
#     # Iterar sobre cada línea del texto
#     for linea in texto.split('\n'):
#         # Buscar coincidencias con las expresiones regulares
#         match_capitulo = regex_capitulo.match(linea)
#         # match_articulo = regex_articulo.match(linea)

#         if "CAPITULO" in linea:
#             print(linea)

#         # Si hay coincidencia con el formato de un capítulo
#         if match_capitulo:
#             if capitulo_actual and articulo_actual and texto_actual:
#                 info.append({
#                     'source': "Estatutos (2022)",
#                     "page": 5,
#                     'capitulo': capitulo_actual.strip(),
#                     'articulo': articulo_actual.strip(),
#                     'contenido': texto_actual.strip()
#                 })
#                 texto_actual = ''

#             # Guardar el capítulo actual y reiniciar el artículo y el texto
#             capitulo_actual = f"CAPÍTULO {match_capitulo.group(1)}: "
#             articulo_actual = None
#             texto_actual = ''
#             print("CAPITULO:", capitulo_actual)
        
#         # Si hay coincidencia con el formato de un artículo
#         elif "Artículo" in linea:
#             if capitulo_actual and articulo_actual and texto_actual:
#                 info.append({
#                     'source': "Estatutos (2022)",
#                     "page": 5,
#                     'capitulo': capitulo_actual.strip(),
#                     'articulo': articulo_actual.strip(),
#                     'contenido': texto_actual.strip()
#                 })
#                 texto_actual = ''

#             # Guardar el artículo actual y reiniciar el texto
#             # articulo_actual = match_articulo.group(1)
#             articulo_actual = linea.strip()
#             # texto_actual = match_articulo.group(2)
#             texto_actual = ''
#             print("ARTICULO:", articulo_actual)
        
#         elif articulo_actual is None and capitulo_actual is not None:
#             capitulo_actual += linea.strip() + ' '
        
#         # Si no hay coincidencia con ninguno de los formatos anteriores, añadir la línea al texto actual
#         else:
#             texto_actual += linea + ' '
        
#         # Si tenemos un capítulo y un artículo actual y un texto, añadirlos a la lista de información
#         # if capitulo_actual and articulo_actual and texto_actual:
#         #     info.append({
#         #         'capitulo': capitulo_actual.strip(),
#         #         'articulo': articulo_actual.strip(),
#         #         'texto': texto_actual.strip()
#         #     })
#         #     texto_actual = ''
    
#     return info

# def extraer_texto_desde_pdf(pdf_path):
#     texto = ''
#     with open(pdf_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         for page_num in range(4, len(reader.pages)):
#             page = reader.pages[page_num]
#             texto += page.extract_text()
#     return texto

# pdf_path = 'documentacion/1. Estatutos (2022).pdf'
# save_path_result = "documentacion/1. Estatutos (2022).json"

# # Extraer texto del PDF
# texto_pdf = extraer_texto_desde_pdf(pdf_path)

# # Extraer información del texto
# informacion = extraer_info_desde_texto(texto_pdf)

# # Convertir la información a formato JSON
# json_info = json.dumps(informacion, indent=4, ensure_ascii=False)

# with open(save_path_result, "w", encoding='utf-8') as json_file:
#     json_file.write(json_info)
# print(json_info)


## VECTOR_DB

In [ ]:
input_path = "documentacion/1. Estatutos (2022).json"
output_path = "vector_db"
reset_db = False

In [ ]:
import json
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import shutil
import torch


class VectorDBGenerator:
    def __init__(self, output_path, reset_db=True):
        self.output_path = output_path
        if reset_db:
            self.__delete_db()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.embeddings = HuggingFaceEmbeddings()

    def generate_vectordb(self, input_path):
        json_data = self.__read_json_file(input_path)
        documents = self.__generate_documents(json_data)
        vectordb = self.__generate_vectors(documents)
        return vectordb
    

    def __read_json_file(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data


    def __generate_documents(self, data):
        splits = []
        for el in data:
            metadata = {}
            metadata["source"] = el["source"]
            metadata["page"] = el["page"]
            metadata["capitulo"] = el["capitulo"]
            metadata["articulo"] = el["articulo"]
            doc =  Document(page_content=el["contenido"], metadata=metadata)
            splits.append(doc)
        return splits
    
    def __generate_vectors(self, documents):
        vectordb = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory=self.output_path
        )
        return vectordb
    
    def __delete_db(self):
        if os.path.exists(self.output_path):
            shutil.rmtree(self.output_path)
        os.makedirs(self.output_path)


In [ ]:
generator = VectorDBGenerator(output_path, reset_db)
generator.generate_vectordb(input_path)

## RAG

In [ ]:
model_id = "../Llama-2-7b-chat-hf"
input_path = "vector_db"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline

In [ ]:
# Load db
vectordb = Chroma(persist_directory=input_path, embedding_function=HuggingFaceEmbeddings())
print(vectordb._collection.count())


# Load model directly
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
model.to(device)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Utiliza los siguientes elementos de contexto para responder a la pregunta del final. Genera una única respuesta. Si no sabes la respuesta, di simplemente que no la sabes, no intentes inventarte una respuesta. Utiliza tres frases como máximo. La respuesta debe ser lo más concisa posible.
{context}
Pregunta: {question}
Respuesta:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
def imprimir_respuesta(result):
    aswer = result["result"]
    print(f"PREGUNTA: {result['query']}")
    print(f"RESPUESTA: {aswer}")
    print("FUENTES:")
    for el in result["source_documents"]:
        print("{" + f"'source': '{el.metadata['source']}', 'page': '{el.metadata['page']}', 'capitulo': '{el.metadata['capitulo']}', 'articulo': '{el.metadata['articulo']}" + "}")
        print(el.page_content)

In [ ]:
question = "Cómo se puede disolver el grupo? Dime los pasos y condiciones"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)

In [ ]:
question = "¿Quien es el Kraal?"
result = qa_chain.invoke({"query": question})

imprimir_respuesta(result)